In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
ticker = "SPY"
data = yf.download(ticker, start = "2010-01-01",end="2022-04-22")


[*********************100%%**********************]  1 of 1 completed


In [3]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,112.370003,113.389999,111.510002,113.330002,86.580727,118944600
2010-01-05,113.260002,113.680000,112.849998,113.629997,86.809944,111579900
2010-01-06,113.519997,113.989998,113.430000,113.709999,86.871056,116074400
2010-01-07,113.500000,114.330002,113.180000,114.190002,87.237747,131091100
2010-01-08,113.889999,114.620003,113.660004,114.570000,87.528053,126402800
...,...,...,...,...,...,...
2022-04-14,443.549988,444.730011,437.679993,437.790009,422.986664,97869500
2022-04-18,436.809998,439.750000,435.609985,437.970001,423.160583,66002500
2022-04-19,437.859985,445.799988,437.679993,445.040009,429.991486,77821000


In [4]:
data["daily_return"] = data["Adj Close"].pct_change()
data["state"] = np.where(data["daily_return"] >=0,"up", "down")

In [5]:
data

,Open,High,Low,Close,Adj Close,Volume,daily_return,state
Date,,,,,,,,
2010-01-04,112.370003,113.389999,111.510002,113.330002,86.580727,118944600,NaN,down
2010-01-05,113.260002,113.680000,112.849998,113.629997,86.809944,111579900,0.002647,up
2010-01-06,113.519997,113.989998,113.430000,113.709999,86.871056,116074400,0.000704,up
2010-01-07,113.500000,114.330002,113.180000,114.190002,87.237747,131091100,0.004221,up
2010-01-08,113.889999,114.620003,113.660004,114.570000,87.528053,126402800,0.003328,up
...,...,...,...,...,...,...,...,...
2022-04-14,443.549988,444.730011,437.679993,437.790009,422.986664,97869500,-0.012452,down
2022-04-18,436.809998,439.750000,435.609985,437.970001,423.160583,66002500,0.000411,up
2022-04-19,437.859985,445.799988,437.679993,445.040009,429.991486,77821000,0.016143,up


In [6]:
up_counts = len(data[data["state"] == "up"])
down_counts = len(data[data["state"] == "down"])
up_to_up = len(data[(data["state"] == "up") & (data["state"].shift(-1) == "up")])/len(data.query('state=="up"'))
down_to_up = len(data[(data["state"] == "up") & (data["state"].shift(-1) == "down")])/len(data.query('state=="up"'))
up_to_down = len(data[(data["state"] == "down") & (data["state"].shift(-1) == "up")])/len(data.query('state=="down"'))
down_to_down = len(data[(data["state"] == "down") & (data["state"].shift(-1) == "down")])/len(data.query('state=="down"'))
transition_matrix = pd.DataFrame({
    "up":[up_to_up, up_to_down],
    "down":[down_to_up, down_to_down]
}, index=["up", "down"])

print(transition_matrix)

            up      down
up    0.546767  0.453233
down  0.575092  0.424176


In [7]:
# Calculate the probability of 5 consecutive "down" days followed by 2 "up" days
prob_5_down = down_to_down ** 4 * down_to_up
prob_2_up_after_5_down = up_to_up

# Total probability of the sequence
prob_sequence = prob_5_down * prob_2_up_after_5_down ** 2

print("Probability of 5 consecutive 'down' days followed by 2 'up' days:", prob_sequence)

Probability of 5 consecutive 'down' days followed by 2 'up' days: 0.004386413819276711
